# Parsing System Files Example
This notebook is intended to show how the parsing of matpower, pss/e and plexos data can be done to integrate it with the PowerSystems platform. 

In [1]:
] activate env; instantiate

Activating environment at `~/Documents/repos/Examples/env/Project.toml`
  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %

In [2]:
] st

    Status `~/Documents/repos/Examples/env/Project.toml`
  [5ae59095] Colors v0.9.6
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.19.4
  [e2685f51] ECOS v0.10.0
  [60bf3e95] GLPK v0.11.4
  [b6b21f68] Ipopt v0.6.0
  [4076af6c] JuMP v0.20.0
  [51fcb6bd] NamedColors v0.2.0
  [f0f68f2c] PlotlyJS v0.12.5
  [91a5bcdd] Plots v0.26.3
  [e690365d] PowerSimulations v0.2.0 #jd/sim_update (https://github.com/NREL/PowerSimulations.jl.git)
  [bcd98974] PowerSystems v0.4.0 #master (https://github.com/NREL/PowerSystems.jl.git)
  [9e3dc215] TimeSeries v0.16.0
  [0f1e0344] WebIO v0.8.11


Also, run `] build PowerSystems` (here or in a REPL) to download data necessary to run this notebook

In [3]:
using PowerSystems
using TimeSeries
const PSY = PowerSystems
const IS = PSY.InfrastructureSystems;

- To disable PowerModels warnings 
  - Edit : PowerModels=>src=>PowerModels.jl :line 21
  - Memento.config!(LOGGER, "warn")  =>  Memento.config!(LOGGER, "error")

In [4]:
base_dir = dirname(dirname(pathof(PowerSystems)))

"/Users/cbarrows/.julia/packages/PowerSystems/RItCb"

## Read Data files to create a to PowerSystems `System` object

### MATPOWER

In [5]:
sys_matpower = PSY.parse_standard_files(joinpath(base_dir, "data/matpower/RTS_GMLC.m"));

┌ Info: extending matpower format with data: areas 3x3
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/matpower.jl:290
┌ Info: extending matpower format with data: gen_name 158x4
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/matpower.jl:290
┌ Info: added zero cost function data for dclines
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/matpower.jl:530
┌ Info: extending matpower format by appending matrix "gen_name" in to "gen"
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/matpower.jl:625
┌ Info: the voltage setpoint on generator 32 does not match the value at bus 207
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/data.jl:1656
┌ Info: the voltage setpoint on generator 29 does not match the value at bus 202
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/dat

┌ Info: simplifying pwl cost on generator 78, [0.0, 0.0, 0.1666667, 0.0, 0.33333329999999994, 0.0, 0.5, 0.0] -> [0.0, 0.0, 0.5, 0.0]
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/data.jl:1805
┌ Info: simplifying pwl cost on generator 81, [0.0, 0.0, 0.1666667, 0.0, 0.33333329999999994, 0.0, 0.5, 0.0] -> [0.0, 0.0, 0.5, 0.0]
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/data.jl:1805
┌ Info: simplifying pwl cost on generator 101, [0.0, 0.0, 0.309, 0.0, 0.618, 0.0, 0.927, 0.0] -> [0.0, 0.0, 0.927, 0.0]
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/data.jl:1805
┌ Info: simplifying pwl cost on generator 105, [0.0, 0.0, 0.16566669999999997, 0.0, 0.3313333, 0.0, 0.49700000000000005, 0.0] -> [0.0, 0.0, 0.49700000000000005, 0.0]
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/data.jl:1805
┌ Info: simplifying pwl cost on generator 109, [0.0,

┌ Info: pwl x value 0.0033333 is outside the bounds 0.0-0.0 on generator 82
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/data.jl:1730
┌ Info: pwl x value 0.0066667 is outside the bounds 0.0-0.0 on generator 82
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/data.jl:1730
┌ Info: pwl x value 0.01 is outside the bounds 0.0-0.0 on generator 82
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/data.jl:1730
┌ Info: simplifying pwl cost on generator 82, [0.0, 0.0, 0.0033333, 0.0, 0.0066667, 0.0, 0.01, 0.0] -> [0.0, 0.0, 0.01, 0.0]
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/data.jl:1805
┌ Info: simplifying pwl cost on generator 79, [0.0, 0.0, 0.1666667, 0.0, 0.33333329999999994, 0.0, 0.5, 0.0] -> [0.0, 0.0, 0.5, 0.0]
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/data.jl:1805
┌ Info: simplifying pwl 

In [6]:
sys_matpower

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,73
2,FixedAdmittance,ElectricLoad <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
3,HVDCLine,DCBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
4,HydroDispatch,HydroGen <: Generator <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,20
5,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,104
6,LoadZones,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
7,PowerLoad,StaticLoad <: ElectricLoad <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,51
8,RenewableDispatch,RenewableGen <: Generator <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,30
9,RenewableFix,RenewableGen <: Generator <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,31


### PSS/E

In [7]:
sys_psse = PSY.parse_standard_files(joinpath(base_dir,"data/psse_raw/RTS-GMLC.RAW"));

┌ Info: The PSS(R)E parser currently supports buses, loads, shunts, generators, branches, transformers, and dc lines
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/common.jl:22
┌ Info: At line 77, unexpected section: expected: BUS, comment specified: BUS
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/pti.jl:578
┌ Info: The following fields in LOAD are missing: INTRPT
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/pti.jl:489
┌ Info: The following fields in LOAD are missing: INTRPT
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/pti.jl:489
┌ Info: The following fields in LOAD are missing: INTRPT
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/pti.jl:489
┌ Info: The following fields in LOAD are missing: INTRPT
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/pti.

┌ Info: At line 490, unexpected section: expected: ZONE, comment specified: ZONE
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/pti.jl:578
┌ Info: At line 491, unexpected section: expected: INTER-AREA TRANSFER, comment specified: INTER-AREA TRANSFER
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/pti.jl:578
┌ Info: At line 493, unexpected section: expected: OWNER, comment specified: OWNER
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/pti.jl:578
┌ Info: At line 494, unexpected section: expected: FACTS CONTROL DEVICE, comment specified: FACTS CONTROL DEVICE
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/pti.jl:578
┌ Info: At line 498, unexpected section: expected: SWITCHED SHUNT, comment specified: SWITCHED SHUNT
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/pti.jl:578
┌ Info: At line 499, unexp

In [8]:
sys_psse

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,73
2,FixedAdmittance,ElectricLoad <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
3,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,105
4,PowerLoad,StaticLoad <: ElectricLoad <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,51
5,TapTransformer,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,15
6,ThermalStandard,ThermalGen <: Generator <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,160


### CDM - Tabular data format adopted by the [RTS-GMLC](https://github.com/gridmod/rts-gmlc)

*This format parses the all of time series data and assignes it to the `PowerSystemRaw` object. To generate a valid `System` we need to select one of the forecasts.*

In [9]:
RTS_GMLC_DIR = joinpath(base_dir,"data/RTS_GMLC");

#parse in tabular data
rawsys = PSY.PowerSystemTableData(RTS_GMLC_DIR,100.0, joinpath(RTS_GMLC_DIR,"user_descriptors.yaml"))

#create an hourly model from tabular data
sys = System(rawsys; forecast_resolution = Dates.Hour(1));

┌ Info: Parsing csv files in Reserves ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:133
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Down.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:137
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Up.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:137
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Down.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:137
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Up.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:137
┌ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R1.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pow

┌ Warning: User-defined column name Voltage Limit Min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:862
┌ Warning: User-defined column name Voltage Limit Max is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:862
┌ Warning: User-defined column name Voltage Limit Min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:862
┌ Warning: User-defined column name Voltage Limit Max is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:862
┌ Warning: User-defined column name Voltage Limit Min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:862
┌ Warning: User-defined column name Voltage Limit Max is not in datafr

┌ Warning: User-defined column name Voltage Limit Min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:862
┌ Warning: User-defined column name Voltage Limit Max is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:862
┌ Warning: User-defined column name Voltage Limit Min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:862
┌ Warning: User-defined column name Voltage Limit Max is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:862
┌ Warning: User-defined column name Voltage Limit Min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:862
┌ Warning: User-defined column name Voltage Limit Max is not in datafr

┌ Warning: User-defined column name From Active Power Min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:862
┌ Warning: User-defined column name From Active Power Max is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:862
┌ Warning: User-defined column name To Active Power Min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:862
┌ Warning: User-defined column name To Active Power Max is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:862
┌ Warning: User-defined column name DC Line Category is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/power_system_table_data.jl:862
┌ Warning: User-defined column name Variable Cost is not in

In [10]:
sys

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,73
2,GenericBattery,Storage <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
3,HVDCLine,DCBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
4,HydroDispatch,HydroGen <: Generator <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,20
5,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,104
6,LoadZones,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
7,PowerLoad,StaticLoad <: ElectricLoad <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,73
8,ProportionalReserve,Reserve <: Service <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,7
9,RenewableDispatch,RenewableGen <: Generator <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,30


## `System`s created from MATPOWER and PSS/E files don't automatically contain time series data

In [11]:
FORECASTS_DIR = joinpath(base_dir,"data/forecasts/")


"/Users/cbarrows/.julia/packages/PowerSystems/RItCb/data/forecasts/"

In [12]:
sys_5 = PSY.parse_standard_files(joinpath(base_dir, "data/matpower", "case5_re.m"))

┌ Info: extending matpower format with data: areas 1x3
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/matpower.jl:290
┌ Info: extending matpower format with data: gen_name 7x4
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/matpower.jl:290
┌ Info: extending matpower format by appending matrix "gen_name" in to "gen"
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/matpower.jl:625
┌ Info: reversing the orientation of branch 6 (4, 3) to be consistent with other parallel branches
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/data.jl:1190
┌ Info: removing 1 cost terms from generator 4: [4000.0, 0.0]
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_io/data.jl:1839
┌ Info: removing 1 cost terms from generator 1: [1400.0, 0.0]
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/RItCb/src/parsers/pm_

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5
2,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5
3,LoadZones,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
4,PhaseShiftingTransformer,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,2
5,PowerLoad,StaticLoad <: ElectricLoad <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
6,RenewableDispatch,RenewableGen <: Generator <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,2
7,ThermalStandard,ThermalGen <: Generator <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5


In [13]:
ts_pointers = IS.read_timeseries_metadata(joinpath(FORECASTS_DIR,
                            "5bus_ts","timeseries_pointers_da.json"))

5-element Array{InfrastructureSystems.TimeseriesFileMetadata,1}:
 InfrastructureSystems.TimeseriesFileMetadata("DAY_AHEAD", "Generator", "SolarBusC", "PMax MW", 1.0, "/Users/cbarrows/.julia/packages/PowerSystems/RItCb/data/forecasts/5bus_ts/gen/Renewable/PV/da_solar5.csv", Float64[], "Deterministic")
 InfrastructureSystems.TimeseriesFileMetadata("REAL_TIME", "Generator", "WindBusA", "PMax MW", 1.0, "/Users/cbarrows/.julia/packages/PowerSystems/RItCb/data/forecasts/5bus_ts/gen/Renewable/WIND/da_wind5.csv", Float64[], "Deterministic")
 InfrastructureSystems.TimeseriesFileMetadata("REAL_TIME", "ElectricLoad", "bus2", "MW Load", 1.0, "/Users/cbarrows/.julia/packages/PowerSystems/RItCb/data/forecasts/5bus_ts/load/da_load5.csv", Float64[], "Deterministic")               
 InfrastructureSystems.TimeseriesFileMetadata("REAL_TIME", "ElectricLoad", "bus3", "MW Load", 1.0, "/Users/cbarrows/.julia/packages/PowerSystems/RItCb/data/forecasts/5bus_ts/load/da_load5.csv", Float64[], "Deterministic")   

In [14]:
add_forecasts!(sys_5, ts_pointers)
sys_5

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5
2,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5
3,LoadZones,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
4,PhaseShiftingTransformer,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,2
5,PowerLoad,StaticLoad <: ElectricLoad <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
6,RenewableDispatch,RenewableGen <: Generator <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,2
7,ThermalStandard,ThermalGen <: Generator <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5
,ConcreteType,SuperTypes,Count
,String,String,Int64


## JSON serialization of data

In [15]:
path, io = mktemp()
@info "Serializing to $path"
to_json(io, sys)
close(io)

┌ Info: Serializing to /var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/jl_6vJurR
└ @ Main In[15]:2


In [16]:
filesize(path)/1000000 #MB

0.421313

In [17]:
sys2 = System(path)

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,73
2,GenericBattery,Storage <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
3,HVDCLine,DCBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
4,HydroDispatch,HydroGen <: Generator <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,20
5,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,104
6,LoadZones,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
7,PowerLoad,StaticLoad <: ElectricLoad <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,73
8,ProportionalReserve,Reserve <: Service <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,7
9,RenewableDispatch,RenewableGen <: Generator <: Injection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,30
